In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
#input_dtypes = ['int32', 'S10'] , dtype=input_dtypes
df = pd.read_csv("Zillow_data_20191023_(Raleigh Durham Cary).csv")

In [3]:
df.head()

,@id,@name,@type,FIPScounty,bathrooms,bedrooms,city,finishedSqFt,lastSoldDate,lastSoldPrice,...,street,taxAssessment,taxAssessmentYear,totalRooms,useCode,yearBuilt,zestimate,zindexValue,zipcode,zpid
0,270325,Southeast Raleigh,neighborhood,37183.0,3.0,3.0,Raleigh,1618.0,4/2/2019,19000.0,...,3915 Alder Grove Ln,28000.0,2018.0,NaN,SingleFamily,2019.0,NaN,"149,300",27610,79886745
1,54047,Raleigh,city,37183.0,3.0,4.0,Raleigh,2537.0,NaN,NaN,...,3207 Britmass Dr,40000.0,2018.0,NaN,SingleFamily,2019.0,NaN,"221,100",27616,215491685
2,343869,Northeast Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,1773.0,12/1/1989,102500.0,...,2004 Carthage Cir,159269.0,2018.0,7.0,SingleFamily,1989.0,224901.0,"171,900",27604,6504197
3,54047,Raleigh,city,37183.0,4.0,3.0,Raleigh,4617.0,5/16/2003,575000.0,...,9924 Waterview Rd,618168.0,2018.0,12.0,SingleFamily,1994.0,869696.0,"221,100",27615,6483992
4,270318,North Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,3066.0,5/27/2004,364000.0,...,2047 Hornbeck Ct,435002.0,2018.0,14.0,SingleFamily,2003.0,469902.0,"270,100",27614,65332699


In [4]:
#### Generate the Data and Target data. Target Data is LAST_SOLD_PRICE. 
#### Rest of the inputs are InputData.

housing_data = df.copy()   ## Make a copy of the data before cleaning.

In [5]:
## Clean the dataset to drop DataPoints where LastSoldPrice is not available.

print(f"Shape of dataset before cleaning is {housing_data.shape}")
housing_data.dropna(subset=['zestimate'], inplace = True)
print(f"Shape of dataset after cleaning is {housing_data.shape}")

Shape of dataset before cleaning is (1683, 25)
Shape of dataset after cleaning is (1594, 25)


In [6]:
housing_data.describe()

,@id,FIPScounty,bathrooms,bedrooms,finishedSqFt,lastSoldPrice,latitude,longitude,lotSizeSqFt,taxAssessment,taxAssessmentYear,totalRooms,yearBuilt,zestimate,zipcode,zpid
count,1594.000000,1405.000000,1589.000000,1589.000000,1593.000000,1.242000e+03,1594.000000,1594.000000,1.491000e+03,1.391000e+03,1396.000000,980.000000,1518.000000,1.594000e+03,1594.000000,1.594000e+03
mean,311732.245922,37140.024911,3.231278,3.697294,2722.768362,3.379510e+05,35.877602,-78.776342,2.327884e+04,3.301183e+05,2017.984241,9.121429,1991.719368,4.699123e+05,27623.569636,2.994352e+08
std,299234.568474,57.835237,1.248111,0.875570,1338.966652,4.003116e+05,0.104047,0.130304,5.623412e+04,2.802546e+05,0.124588,39.792651,25.185985,3.059054e+05,71.687343,6.574938e+08
min,24457.000000,37037.000000,1.000000,1.000000,544.000000,2.500000e+03,35.594410,-79.033768,8.710000e+02,4.800000e+03,2017.000000,2.000000,1888.000000,7.552600e+04,27511.000000,6.375533e+06
25%,51297.000000,37063.000000,2.500000,3.000000,1811.000000,1.620000e+05,35.801305,-78.893846,7.448500e+03,1.658070e+05,2018.000000,6.000000,1980.000000,2.907140e+05,27603.000000,6.533989e+06
50%,270318.000000,37183.000000,3.000000,4.000000,2485.000000,2.655000e+05,35.876860,-78.799619,1.136900e+04,2.592100e+05,2018.000000,8.000000,1998.000000,3.906235e+05,27614.000000,5.002811e+07
75%,343876.000000,37183.000000,4.000000,4.000000,3272.000000,3.990000e+05,35.956491,-78.665335,2.003700e+04,4.019795e+05,2018.000000,10.000000,2009.000000,5.537910e+05,27703.000000,9.821294e+07
max,816439.000000,37183.000000,10.000000,7.000000,12174.000000,8.528500e+06,36.130052,-78.454965,1.176120e+06,2.741296e+06,2018.000000,1250.000000,2019.000000,3.695643e+06,27713.000000,2.141154e+09


In [7]:
print(f"Maximum Estimated price is {housing_data['zestimate'].max()}")
print(f"Minimum Estimated price is {housing_data['zestimate'].min()}")

Maximum Estimated price is 3695643.0
Minimum Estimated price is 75526.0


In [8]:
housing_data['zestimate'].describe()['25%']

290714.0

In [9]:
#### To implement classification divide the input data into 5 clusters.
#### We will try to predict what group a home in given region would fall into

bin_labels = ['1', '2', '3', '4', '5']
housing_data['price_bin'] = pd.qcut(housing_data['zestimate'], q=[0, .2, .4, .6, .8, 1], labels = bin_labels)

In [10]:
housing_data.head()

,@id,@name,@type,FIPScounty,bathrooms,bedrooms,city,finishedSqFt,lastSoldDate,lastSoldPrice,...,taxAssessment,taxAssessmentYear,totalRooms,useCode,yearBuilt,zestimate,zindexValue,zipcode,zpid,price_bin
2,343869,Northeast Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,1773.0,12/1/1989,102500.0,...,159269.0,2018.0,7.0,SingleFamily,1989.0,224901.0,"171,900",27604,6504197,1
3,54047,Raleigh,city,37183.0,4.0,3.0,Raleigh,4617.0,5/16/2003,575000.0,...,618168.0,2018.0,12.0,SingleFamily,1994.0,869696.0,"221,100",27615,6483992,5
4,270318,North Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,3066.0,5/27/2004,364000.0,...,435002.0,2018.0,14.0,SingleFamily,2003.0,469902.0,"270,100",27614,65332699,4
5,343864,West Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,2979.0,7/31/2007,381500.0,...,343669.0,2018.0,8.0,SingleFamily,1974.0,442602.0,"173,400",27606,6382773,3
6,270318,North Raleigh,neighborhood,37183.0,4.0,4.0,Raleigh,4977.0,7/8/2014,665000.0,...,673517.0,2018.0,8.0,SingleFamily,1997.0,760869.0,"270,100",27615,6551268,5


In [11]:
###### Drop the lastSoldPrice as the column was needed only to generate
###### the bins. SVM is a classification model and not regression.
ml_house_prices = housing_data.copy()
ml_house_prices.drop("zestimate", axis = 1, inplace = True)
ml_house_prices.head()

,@id,@name,@type,FIPScounty,bathrooms,bedrooms,city,finishedSqFt,lastSoldDate,lastSoldPrice,...,street,taxAssessment,taxAssessmentYear,totalRooms,useCode,yearBuilt,zindexValue,zipcode,zpid,price_bin
2,343869,Northeast Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,1773.0,12/1/1989,102500.0,...,2004 Carthage Cir,159269.0,2018.0,7.0,SingleFamily,1989.0,"171,900",27604,6504197,1
3,54047,Raleigh,city,37183.0,4.0,3.0,Raleigh,4617.0,5/16/2003,575000.0,...,9924 Waterview Rd,618168.0,2018.0,12.0,SingleFamily,1994.0,"221,100",27615,6483992,5
4,270318,North Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,3066.0,5/27/2004,364000.0,...,2047 Hornbeck Ct,435002.0,2018.0,14.0,SingleFamily,2003.0,"270,100",27614,65332699,4
5,343864,West Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,2979.0,7/31/2007,381500.0,...,4404 Driftwood Dr,343669.0,2018.0,8.0,SingleFamily,1974.0,"173,400",27606,6382773,3
6,270318,North Raleigh,neighborhood,37183.0,4.0,4.0,Raleigh,4977.0,7/8/2014,665000.0,...,421 Chatterson Dr,673517.0,2018.0,8.0,SingleFamily,1997.0,"270,100",27615,6551268,5


In [12]:
target = ml_house_prices['price_bin']
target_names = bin_labels

In [13]:
##### Drop the price_bin as that is the prediction target. 

ml_house_prices.drop("price_bin", axis = 1, inplace = True)
ml_house_prices.head()

,@id,@name,@type,FIPScounty,bathrooms,bedrooms,city,finishedSqFt,lastSoldDate,lastSoldPrice,...,state,street,taxAssessment,taxAssessmentYear,totalRooms,useCode,yearBuilt,zindexValue,zipcode,zpid
2,343869,Northeast Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,1773.0,12/1/1989,102500.0,...,NC,2004 Carthage Cir,159269.0,2018.0,7.0,SingleFamily,1989.0,"171,900",27604,6504197
3,54047,Raleigh,city,37183.0,4.0,3.0,Raleigh,4617.0,5/16/2003,575000.0,...,NC,9924 Waterview Rd,618168.0,2018.0,12.0,SingleFamily,1994.0,"221,100",27615,6483992
4,270318,North Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,3066.0,5/27/2004,364000.0,...,NC,2047 Hornbeck Ct,435002.0,2018.0,14.0,SingleFamily,2003.0,"270,100",27614,65332699
5,343864,West Raleigh,neighborhood,37183.0,3.0,4.0,Raleigh,2979.0,7/31/2007,381500.0,...,NC,4404 Driftwood Dr,343669.0,2018.0,8.0,SingleFamily,1974.0,"173,400",27606,6382773
6,270318,North Raleigh,neighborhood,37183.0,4.0,4.0,Raleigh,4977.0,7/8/2014,665000.0,...,NC,421 Chatterson Dr,673517.0,2018.0,8.0,SingleFamily,1997.0,"270,100",27615,6551268


In [14]:
###### Drop the additional columns, that we dont need for training.
###### Since we are dropping columns from DataFrame, make a copy of that dataframe before drop.

##print(ml_house_prices.columns)
ml_house_prices_2 = ml_house_prices.copy()
ml_house_prices_2.drop(["@id", "@name", "FIPScounty", "city", "lastSoldDate", "lastSoldPrice", "links", 'state', 'street', 'taxAssessmentYear', 'useCode', 'zindexValue', 'zipcode', 'zpid'], axis=1, inplace=True)

####### Print a list of all columns in the dataframe.
print(ml_house_prices_2.columns)

Index(['@type', 'bathrooms', 'bedrooms', 'finishedSqFt', 'latitude',
       'longitude', 'lotSizeSqFt', 'taxAssessment', 'totalRooms', 'yearBuilt'],
      dtype='object')


In [15]:
ml_house_prices_2["@type"] = pd.Categorical(ml_house_prices_2["@type"])
ml_house_prices_2["type-codes"] = ml_house_prices_2["@type"].cat.codes
ml_house_prices_2.drop(['@type'], axis = 1, inplace = True)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ml_house_prices_2, target, random_state = 42)

In [17]:
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)

In [18]:
print(f"Model Fit activities started at {dt.now().strftime('%d-%b-%Y %I:%M:%S %p')}")

Model Fit activities started at 26-Oct-2019 11:53:27 PM


In [ ]:
start_date_time = dt.now()
print(f"Model fit activities started at {start_date_time.strftime('%d-%b-%Y %I:%M:%S %p')}")
from sklearn.svm import SVC
model = SVC(kernel = 'poly', random_state = 42)
model.fit (X_train, y_train)
end_date_time = dt.now()
print(f"Model fit ended at {end_date_time.strftime('%d-%b-%Y %I:%M:%S %p')}")

Model fit activities started at 26-Oct-2019 11:53:28 PM


C:\Users\NandK\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
